# Double Pulse Generator

Control the PIO-based double pulse generator that runs on the RP Pico W firmware. This workflow produces a pair of pulses with a configurable separation (500 ns to 500 µs) and repetition rate so you can measure HDSoC deadtime.

## Imports

The examples below use the serial command sender, but the `WebCommandSender` can be substituted if you start the on-board webserver.

In [1]:
import sys
import os
import time

notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'source')))

from serial_controller import SerialController
from serial_command_sender import SerialCommandSender
from web_controller import WebController
from web_command_sender import WebCommandSender


## Configure the double pulse generator

Update `SERIAL_PORT`, `hardware_id`, and the `settings` dictionary in the command-listing cell below. Running it sends `list_commands` so you can confirm connectivity before creating or tuning pulses.

In [ ]:
SERIAL_PORT = "/dev/ttyACM0"  # Replace with the Pico W serial port on your system
hardware_id = "double_pulse_gen"

settings = {
    'pin_number': 15,
    'repetition_rate_hz': 5000,
    'pulse_width_ns': 500,
    'separation_ns': 2000,
    'start_on_init': False
}

serial_controller = SerialController(port=SERIAL_PORT, debug=False)
serial_controller.connect()
serial_command_sender = SerialCommandSender(serial_controller)

try:
    commands = serial_command_sender.send_command('list_commands')
    print('Available commands:' + commands)
finally:
    serial_controller.close()


## Create and tune the double pulse waveform

Once the quick command test runs cleanly, execute the cell below to create the double pulse hardware block and adjust `settings`. Updates to rate or separation are applied live.

In [ ]:
serial_controller = SerialController(port=SERIAL_PORT, debug=False)
serial_controller.connect()
serial_command_sender = SerialCommandSender(serial_controller)

try:
    print(serial_command_sender.send_command('create', 'double_pulse', settings, hardware_id))
    print(serial_command_sender.send_command('start', hardware_id))

    for separation in (700, 1500, 2500):
        response = serial_command_sender.send_command(
            'apply_hardware_settings',
            hardware_id,
            {'separation_ns': separation}
        )
        print(f'Separation set to {separation} ns -> {response}')
        time.sleep(1)

    for rate in (1000, 10000):
        response = serial_command_sender.send_command(
            'apply_hardware_settings',
            hardware_id,
            {'repetition_rate_hz': rate}
        )
        print(f'Repetition rate set to {rate} Hz -> {response}')
        time.sleep(1)

    time.sleep(3)
    print(serial_command_sender.send_command('stop', hardware_id))
    print(serial_command_sender.send_command('delete', hardware_id))
finally:
    serial_controller.close()


### Wi-Fi control

To use Wi-Fi instead of serial, start the webserver on the Pico, create a `WebController(base_url='http://<ip>:<port>', debug=True)` and pass it to `WebCommandSender`. The command usage stays the same: `web_command_sender.send_command('create', 'double_pulse', settings, hardware_id)`.